<a href="https://colab.research.google.com/github/haneeth25/Titanic---Machine-Learning-from-Disaster/blob/main/Titanic_more_acurate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##First Set all the csv in your google colab notebook

> The main moto is to predict what sort of people are more likely to survive 

[This problem is from kaggle (click here to see the question)](https://www.kaggle.com/c/titanic)

## Now let us import all basic requirements like Numpy , Pandas , Matplotlib .

In [144]:
%matplotlib inline 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [145]:
#Let us import our train.csv as df 
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Here in the train data set we have some columns which are not required like -> PassengerId , Name , Ticket("basically ticket depend on fair so here we don’t need the ticket column) , Cabin , Embarked .

we will now drop all these columns 



#Here we need to modify the data in such a way that we can use it

In [146]:
x = df.drop(["PassengerId","Survived","Name","Ticket","Cabin","Embarked"],axis = 1)
x

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,22.0,1,0,7.2500
1,1,female,38.0,1,0,71.2833
2,3,female,26.0,0,0,7.9250
3,1,female,35.0,1,0,53.1000
4,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000
887,1,female,19.0,0,0,30.0000
888,3,female,NaN,1,2,23.4500
889,1,male,26.0,0,0,30.0000


In [147]:
y = df["Survived"]
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

> So now our both x and y labels are ready 

* First we need to check wheter we had any missing data in x or not

In [148]:
x.isna().sum()

Pclass      0
Sex         0
Age       177
SibSp       0
Parch       0
Fare        0
dtype: int64

> Here in age column we have 177 missing values , so now we need to fill that missing column with mean of remaing columns .

In [149]:
x["Age"].fillna(x["Age"].mean(),inplace = True)
x.isna().sum()

Pclass    0
Sex       0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64

> So here we had successfully filled the missing values , now we have to check the type of data we have and we need to convert then into int type so that our model can understand 

In [150]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    object 
 2   Age     891 non-null    float64
 3   SibSp   891 non-null    int64  
 4   Parch   891 non-null    int64  
 5   Fare    891 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 41.9+ KB


> Here we have sex column as object type so we need to convert it into int type

In [151]:
# To convert it we will use oneHotEncoder and columnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 

obj_type_columns = ["Sex"]

#initiate oneHotEncoder
one_hot = OneHotEncoder()

transformer = ColumnTransformer([("one_hot" , one_hot,obj_type_columns)],
                                remainder= "passthrough" )

changed_x =  transformer.fit_transform(x)
changed_x = pd.DataFrame(changed_x)
changed_x

,0,1,2,3,4,5,6
0,0.0,1.0,3.0,22.000000,1.0,0.0,7.2500
1,1.0,0.0,1.0,38.000000,1.0,0.0,71.2833
2,1.0,0.0,3.0,26.000000,0.0,0.0,7.9250
3,1.0,0.0,1.0,35.000000,1.0,0.0,53.1000
4,0.0,1.0,3.0,35.000000,0.0,0.0,8.0500
...,...,...,...,...,...,...,...
886,0.0,1.0,2.0,27.000000,0.0,0.0,13.0000
887,1.0,0.0,1.0,19.000000,0.0,0.0,30.0000
888,1.0,0.0,3.0,29.699118,1.0,2.0,23.4500
889,0.0,1.0,1.0,26.000000,0.0,0.0,30.0000


In [152]:
changed_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       891 non-null    float64
 1   1       891 non-null    float64
 2   2       891 non-null    float64
 3   3       891 non-null    float64
 4   4       891 non-null    float64
 5   5       891 non-null    float64
 6   6       891 non-null    float64
dtypes: float64(7)
memory usage: 48.9 KB


> So finally we had converted all columns into int type

# we had modified the complete data so now we can start creating the machine learning model

> First we need to split our data into train and test data 

In [153]:
from sklearn.model_selection import train_test_split 
x_train , x_test , y_train , y_test = train_test_split(changed_x , y , test_size = 0.2)

In [154]:
x_train

,0,1,2,3,4,5,6
344,0.0,1.0,2.0,36.0,0.0,0.0,13.0000
312,1.0,0.0,2.0,26.0,1.0,1.0,26.0000
327,1.0,0.0,2.0,36.0,0.0,0.0,13.0000
60,0.0,1.0,3.0,22.0,0.0,0.0,7.2292
321,0.0,1.0,3.0,27.0,0.0,0.0,7.8958
...,...,...,...,...,...,...,...
489,0.0,1.0,3.0,9.0,1.0,1.0,15.9000
772,1.0,0.0,2.0,57.0,0.0,0.0,10.5000
378,0.0,1.0,3.0,20.0,0.0,0.0,4.0125
288,0.0,1.0,2.0,42.0,0.0,0.0,13.0000


In [155]:
y_train

344    0
312    0
327    1
60     0
321    0
      ..
489    1
772    0
378    0
288    1
418    0
Name: Survived, Length: 712, dtype: int64

In [156]:
x_test

,0,1,2,3,4,5,6
428,0.0,1.0,3.0,29.699118,0.0,0.0,7.7500
532,0.0,1.0,3.0,17.000000,1.0,1.0,7.2292
388,0.0,1.0,3.0,29.699118,0.0,0.0,7.7292
107,0.0,1.0,3.0,29.699118,0.0,0.0,7.7750
423,1.0,0.0,3.0,28.000000,1.0,1.0,14.4000
...,...,...,...,...,...,...,...
198,1.0,0.0,3.0,29.699118,0.0,0.0,7.7500
843,0.0,1.0,3.0,34.500000,0.0,0.0,6.4375
600,1.0,0.0,2.0,24.000000,2.0,1.0,27.0000
299,1.0,0.0,1.0,50.000000,0.0,1.0,247.5208


In [157]:
y_test

428    0
532    0
388    0
107    1
423    0
      ..
198    1
843    0
600    1
299    1
20     0
Name: Survived, Length: 179, dtype: int64

> We had successfully created the train and test data set now we will use LogisticRegression , KNeighborsClassifier , RandomForestClassifier ,to find which model gives most accurate values

In [158]:
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier 
# put all these models into a dictionary 

models = {
    "logistic regression" : LogisticRegression(),
    "KNN" : KNeighborsClassifier(),
    "random forest classifier" : RandomForestClassifier()
}

def fit_and_score(models,x_train,x_test,y_train,y_test):
  """
  fits and evaluates given machine learning models.
  models: a dict of differnt sklearn models
  x_train:training data
  x_test:tresting data
  Y_train:traing data
  y_test:testing data
  """
  #set random seed
  np.random.seed(42)
  #make a dictionary to keep model scores 
  model_scores = {}

  #loop through models 
  for name, model in models.items():
    #fit the model to data 
    model.fit(x_train,y_train)
    #evaluate the model and append its score to model_scores
    model_scores[name] = model.score(x_test,y_test)
  return model_scores

In [159]:
model_scores = fit_and_score(models,
                             x_train,
                             x_test,
                             y_train,
                             y_test)
model_scores

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'KNN': 0.7262569832402235,
 'logistic regression': 0.8659217877094972,
 'random forest classifier': 0.8379888268156425}

## So here we have logistic regression accuracy is 0.8379 and random forest classifer accuracy is 0.7932 so now we will tune this and find the best parameters for which this provide more acurate score

In [160]:
#create a hyperparameter grid for logistic regression

log_reg_grid = {"C":np.logspace(-4,4,30),
                "solver" : ["liblinear"]} 

#create a hyperparameter grid for randomforest classifier
rf_grid = {"n_estimators" : np.arange(10,100,50),
           "max_depth" : [None,3,5,10],
           "min_samples_split" : np.arange(2,20,2),
           "min_samples_leaf" : np.arange(1,20,2)}

In [161]:
# Tune logistic regression 
np.random.seed(42)
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
logistic_tuning = RandomizedSearchCV(LogisticRegression(),
                                     param_distributions = log_reg_grid,
                                     cv = 5,
                                     n_iter = 500,
                                     verbose = True)

logistic_tuning.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 30 is smaller than n_iter=500. Running 30 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.6s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='deprecated', n_iter=500, n_jobs=None,
                   param_distributions={'C'...
       2.04335972e-01, 3.85662042e-01, 7.27895384e-01, 1.37382380e+00,
       2.59294380e+00, 4.89390092e+00, 9.23670857e+00, 1.74332882e+01,
       3.29034456e+01, 6.21016942e+01, 1.17210230e+02, 2.21221629e+02,
       4.17531894e+0

In [162]:
logistic_tuning.score(x_test,y_test)

0.8435754189944135

In [163]:
#Tune random forest classifier
np.random.seed(42)
rfc_tuning = RandomizedSearchCV(RandomForestClassifier(),
                                param_distributions = rf_grid,
                                cv = 5,
                                n_iter = 400,
                                verbose = True)

rfc_tuning.fit(x_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    5.4s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [164]:
rfc_tuning.score(x_test,y_test)

0.8547486033519553

## As logistic regression provide more accuracy we will do GridSearchCv on Logistic Regression

In [165]:
np.random.seed(42)
logistic_tuning_grid = GridSearchCV(LogisticRegression(),
                                     param_grid = log_reg_grid,
                                     cv = 5,
                                     verbose = True)

logistic_tuning_grid.fit(x_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.6s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.00000000e-04, 1.8...
       2.04335972e-01, 3.85662042e-01, 7.27895384e-01, 1.37382380e+00,
       2.59294380e+00, 4.89390092e+00, 9.23670857e+00, 1.74332882e+01,
       3.29034456e+01, 6.21016942e+01, 1.17210230e+02, 2.21221629e+02,
       4.17531894e+02, 7.88046282e+02, 1.48735211e+03, 2.80721620e+03,
       5

In [166]:
logistic_tuning_grid.score(x_test,y_test)

0.8435754189944135

## so finally now we will use logistic regresion to predict answers for the test set given by kaggle.com

In [167]:
#first import the test set 
test_set = pd.read_csv("test.csv")
test_set.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


> Now we need to modify the test set in form of our train set then only the model can predict the answers for that we need to remove the unnecessary columns

In [168]:
test_set_x = x = test_set.drop(["PassengerId","Name","Ticket","Cabin","Embarked"],axis = 1)
test_set_x

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,34.5,0,0,7.8292
1,3,female,47.0,1,0,7.0000
2,2,male,62.0,0,0,9.6875
3,3,male,27.0,0,0,8.6625
4,3,female,22.0,1,1,12.2875
...,...,...,...,...,...,...
413,3,male,NaN,0,0,8.0500
414,1,female,39.0,0,0,108.9000
415,3,male,38.5,0,0,7.2500
416,3,male,NaN,0,0,8.0500


In [169]:
#check the missing data from test_set_x
test_set_x.isna().sum()

Pclass     0
Sex        0
Age       86
SibSp      0
Parch      0
Fare       1
dtype: int64

In [170]:
# now we need to fill those age and fare columns with their mean values
test_set_x["Age"].fillna(test_set_x["Age"].mean(),inplace = True)
test_set_x["Fare"].fillna(test_set_x["Fare"].mean(),inplace = True)
test_set_x.isna().sum()

Pclass    0
Sex       0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64

In [171]:
#now we need to convert sex column to int type
# To convert it we will use oneHotEncoder and columnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 

obj_type_columns = ["Sex"]

#initiate oneHotEncoder
one_hot = OneHotEncoder()

transformer = ColumnTransformer([("one_hot" , one_hot,obj_type_columns)],
                                remainder= "passthrough" )

changed_test_set_x =  transformer.fit_transform(test_set_x)
changed_test_set_x = pd.DataFrame(changed_test_set_x)
changed_test_set_x

,0,1,2,3,4,5,6
0,0.0,1.0,3.0,34.50000,0.0,0.0,7.8292
1,1.0,0.0,3.0,47.00000,1.0,0.0,7.0000
2,0.0,1.0,2.0,62.00000,0.0,0.0,9.6875
3,0.0,1.0,3.0,27.00000,0.0,0.0,8.6625
4,1.0,0.0,3.0,22.00000,1.0,1.0,12.2875
...,...,...,...,...,...,...,...
413,0.0,1.0,3.0,30.27259,0.0,0.0,8.0500
414,1.0,0.0,1.0,39.00000,0.0,0.0,108.9000
415,0.0,1.0,3.0,38.50000,0.0,0.0,7.2500
416,0.0,1.0,3.0,30.27259,0.0,0.0,8.0500


In [172]:
#Now finally our test set is ready and we need to predict solution for this test set
y_preds = logistic_tuning_grid.predict(changed_test_set_x)
y_preds

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [173]:
# Now finally we need to convert our data as mentioned in kaggle.com and sumbit it there

final = pd.DataFrame()
final["PassengerId"] = test_set["PassengerId"]
final["Survived"] = pd.DataFrame(y_preds)
final

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [174]:
from google.colab import files
final.to_csv('final.csv',mode = 'w', index=False) 
files.download('final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>